In [ ]:
import numpy as np
import SimpleITK as sitk
import sys
import itk
from itk import TubeTK as ttk
import os
from dataclasses import dataclass
from glob import glob
from typing import List
import pandas as pd
import numpy as np
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.pyplot as plt
import re
import concurrent.futures
import logging
import time
from tqdm import tqdm
import math

logging.basicConfig(level=logging.INFO)

# ********************************
# *                              *
# *     :-)                      *
# *      |   Enter your base     *
# *     / \   directory here!    *
# *                              *
# ********************************
#           |
#           V
#base_directory = "C:/Users/theod/Downloads/esha_topology_final_data/data/"


# *******************************
# *    /\_/\                    *
# *   / o o \                   * 
# *  (   "   )  Enter your max  *
# *   \~(*)~/    workers here!  * 
# *    - ^ -                    *
# *******************************
#           |
#           V
max_workers = 2


: 

In [2]:
@dataclass
class FileSet:
    mra_t1: str
    mra_angio: str
    n4_brain_mask: str
    n4_mask: str

    def get_base_name(self) -> str:
        for file in [self.mra_t1, self.mra_angio, self.n4_brain_mask, self.n4_mask]:
            match = re.match(r"(sub-\d+_ses-\d+)_", os.path.basename(file))
            if match:
                return match.group(1)
        return None

In [3]:
@dataclass
class ExtendedFileSet(FileSet):
    im2iso_n4_brain_mask: str

In [4]:
@dataclass
class VessFileSet:
    im1vess: str
    im1BrainVess: str

    def get_base_name(self) -> str:
        for file in [self.im1vess, self.im1BrainVess]:
            match = re.match(r"(sub-\d+_ses-\d+)_", os.path.basename(file))
            if match:
                return match.group(1)
        return None

In [5]:
def find_files(base_directory: str, pattern: str) -> List[str]:
    return glob(os.path.join(base_directory, pattern), recursive=True)

In [6]:
file_patterns = {
    "MRA_T1": "MRA/sub-*/ses-*/anat/*_T1w.nii.gz",
    "MRA_ANGIO": "MRA/sub-*/ses-*/anat/*_angio.nii.gz",
    "N4_BRAIN_MASK": "N4_bfc/sub-*/ses-*/anat/*_desc-angio_N4bfc_brain_mask.nii.gz",
    "N4_MASK": "N4_bfc/sub-*/ses-*/anat/*_desc-angio_N4bfc_mask.nii.gz",
}

In [7]:
def get_filtered_file_sets(base_directory, file_patterns, output_dir):
    files = {}
    for key, pattern in file_patterns.items():
        files[key] = find_files(base_directory, pattern)

    base_names = set([os.path.basename(f).split("_T1w")[0] for f in files["MRA_T1"]])

    file_sets = []
    for base_name in base_names:
        file_set = FileSet(
            mra_t1=next(f for f in files["MRA_T1"] if base_name in f),
            mra_angio=next(f for f in files["MRA_ANGIO"] if base_name in f),
            n4_brain_mask=next(f for f in files["N4_BRAIN_MASK"] if base_name in f),
            n4_mask=next(f for f in files["N4_MASK"] if base_name in f),
        )
        file_sets.append(file_set)

    file_sets.sort(key=lambda x: x.get_base_name())

    # Remove file sets with existing output files to allow reruns
    # in the event of errors to regenerate the failed data sets
    filtered_file_sets = []
    for file_set in file_sets:
        output_file = os.path.join(output_dir, f"{file_set.get_base_name()}_MRA-VesselEnhanced_patient.tre")
        if not os.path.exists(output_file):
            filtered_file_sets.append(file_set)
            logging.info(file_set)
        else:
            logging.info(f"Skipping file set with base name {file_set.get_base_name()} as the output file already exists")

    return filtered_file_sets

In [8]:
def convert_and_save_images(file_set: FileSet, output_dir: str) -> FileSet:
    
    def convert_single_image_and_save(file_path: str, suffix: str) -> str:
        # Read the input NIfTI file
        image = sitk.ReadImage(file_path)

        # Create the output MHA file path
        base_name = file_set.get_base_name()
        new_file_path = os.path.join(output_dir, f"{base_name}_{suffix}.mha")

        # Write the output MHA file
        sitk.WriteImage(image, new_file_path)

        return new_file_path

    # Convert and save all images in the FileSet
    new_n4_mask = convert_single_image_and_save(file_set.n4_mask, "desc-angio_N4bfc_mask") #mra.mha / mha_mra
    new_mra_t1 = convert_single_image_and_save(file_set.mra_t1, "T1w") #mha_mra_T1 / mha_mri_T1
    new_mra_angio = convert_single_image_and_save(file_set.mra_angio, "angio") #flash.mha / mha_flash
    new_n4_brain_mask = convert_single_image_and_save(file_set.n4_brain_mask, "desc-angio_N4bfc_brain_mask")#flash-Brain.mha / mha_flash_Brain

    # Return an updated FileSet object with the new file paths
    return FileSet(
        n4_mask=new_n4_mask,
        mra_t1=new_mra_t1,
        mra_angio=new_mra_angio,
        n4_brain_mask=new_n4_brain_mask
    )

In [9]:
def convert_files_step2(file_set: FileSet, output_dir: str) -> ExtendedFileSet:
    base_name = file_set.get_base_name()

    # Set output file paths for the new MHA files
    new_n4_brain_mask = os.path.join(output_dir, f"{base_name}_MRMask-Brain_Step2.mha")
    new_mra_angio = os.path.join(output_dir, f"{base_name}_MRA-Iso_Step2.mha")
    new_mra_t1 = os.path.join(output_dir, f"{base_name}_MRT1-Iso_Step2.mha")
    new_n4_mask = os.path.join(output_dir, f"{base_name}_MRA-Brain_Step2.mha")
    new_im2iso_n4_brain_mask = os.path.join(output_dir, f"{base_name}_MRT1-Brain_Step2.mha")

    ImageType = itk.Image[itk.F, 3]
    ReaderType = itk.ImageFileReader[ImageType]
    
    reader1 = ReaderType.New(FileName=file_set.n4_mask)
    reader1.Update()
    im1 = reader1.GetOutput()
    del reader1
    os.remove(file_set.n4_mask)

    reader2 = ReaderType.New(FileName=file_set.mra_t1)
    reader2.Update()
    im2 = reader2.GetOutput()
    del reader2
    os.remove(file_set.mra_t1)
    
    res = ttk.ResampleImage.New(Input = im1) 
    res.SetMakeHighResIso(True)
    res.Update()
    im1iso = res.GetOutput()
    
    imReg = ttk.RegisterImages[ImageType].New() 
    imReg.SetFixedImage(im1iso)
    imReg.SetMovingImage(im2)
    imReg.SetReportProgress(True)
    imReg.SetExpectedOffsetMagnitude(40)
    imReg.SetExpectedRotationMagnitude(0.01)
    imReg.SetExpectedScaleMagnitude(0.01)
    imReg.SetRigidMaxIterations(500)
    imReg.SetRigidSamplingRatio(0.1)
    imReg.SetRegistration("RIGID")
    imReg.SetMetric("MATTES_MI_METRIC")
    imReg.Update()
    im2iso = imReg.GetFinalMovingImage()

    reader = ReaderType.New(FileName=file_set.mra_angio)
    reader.Update()
    imBase = reader.GetOutput()
    del reader
    os.remove(file_set.mra_angio)
  
    reader = ReaderType.New(FileName=file_set.n4_brain_mask)
    reader.Update()
    imBaseB = reader.GetOutput()
    del reader
    os.remove(file_set.n4_brain_mask)
    
    imMath = ttk.ImageMath.New(Input=im2iso)
    imMath.Blur(2)
    im2isoBlur = imMath.GetOutput()
    regB = []
    regBB = []
    imMath.SetInput(imBase)
    imMath.Blur(2)
    imBaseBlur = imMath.GetOutput()
    regBTo1 = ttk.RegisterImages[ImageType].New(FixedImage=im2isoBlur, MovingImage=imBaseBlur)
    regBTo1.SetReportProgress(True)
    regBTo1.SetExpectedOffsetMagnitude(40)
    regBTo1.SetExpectedRotationMagnitude(0.01)
    regBTo1.SetExpectedScaleMagnitude(0.1)
    regBTo1.SetRigidMaxIterations(500)
    regBTo1.SetAffineMaxIterations(500)
    regBTo1.SetRigidSamplingRatio(0.05)
    regBTo1.SetAffineSamplingRatio(0.05)
    regBTo1.SetInitialMethodEnum("INIT_WITH_IMAGE_CENTERS")
    regBTo1.SetRegistration("PIPELINE_AFFINE")
    regBTo1.SetMetric("MATTES_MI_METRIC")
    regBTo1.Update()
    img = regBTo1.ResampleImage("LINEAR", imBase)
    regB.append( img )
    img = regBTo1.ResampleImage("LINEAR", imBaseB)
    regBB.append( img )
    N=1
    regBBT = []
    for i in range(0,N):
        imMath = ttk.ImageMath[itk.Image[itk.F,3]].New(Input=regBB[i])
        imMath.Threshold(0,1,0,1)
        img = imMath.GetOutput()
        if i==0:
            imMathSum = ttk.ImageMath[itk.Image[itk.F,3]].New( img )
            imMathSum.AddImages( img, 1.0/N, 0 )
            sumBBT = imMathSum.GetOutput()
        else:
            imMathSum = ttk.ImageMath[itk.Image[itk.F,3]].New( sumBBT )
            imMathSum.AddImages( img, 1, 1.0/N )
            sumBBT = imMathSum.GetOutput()
    insideMath = ttk.ImageMath[itk.Image[itk.F,3]].New( Input = sumBBT )
    insideMath.Threshold(1,1,1,0)
    insideMath.Erode(5,1,0)
    brainInside = insideMath.GetOutput()

    outsideMath = ttk.ImageMath[itk.Image[itk.F,3]].New( Input = sumBBT )
    outsideMath.Threshold(0,0,1,0)
    outsideMath.Erode(10,1,0)
    brainOutsideAll = outsideMath.GetOutput()
    outsideMath.Erode(20,1,0)
    outsideMath.AddImages(brainOutsideAll, -1, 1)
    brainOutside = outsideMath.GetOutput()

    outsideMath.AddImages(brainInside,1,2)
    brainCombinedMask = outsideMath.GetOutputUChar()

    outsideMath.AddImages(im2iso, 512, 1)
    brainCombinedMaskView = outsideMath.GetOutput()
    LabelMapType = itk.Image[itk.UC,3]
    segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[ImageType,LabelMapType].New()
    segmenter.SetFeatureImage( im1iso )
    segmenter.AddFeatureImage( im2iso )
    segmenter.SetInputLabelMap( brainCombinedMask )
    segmenter.SetObjectId( 2 )
    segmenter.AddObjectId( 1 )
    segmenter.SetVoidId( 0 )
    segmenter.SetErodeDilateRadius( 5 )
    segmenter.Update()
    segmenter.ClassifyImages()
    brainCombinedMaskClassified = segmenter.GetOutputLabelMap()

    cast = itk.CastImageFilter[LabelMapType, ImageType].New()
    cast.SetInput(brainCombinedMaskClassified)
    cast.Update()
    brainMaskF = cast.GetOutput()

    brainMath = ttk.ImageMath[ImageType].New(Input = brainMaskF)
    brainMath.Threshold(2,2,1,0)
    brainMath.Dilate(2,1,0)
    brainMaskD = brainMath.GetOutput()
    brainMath.SetInput( im1 )
    brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0)
    brain = brainMath.GetOutput()
    

    # Save the new MHA files
    itk.imwrite(brainMaskD, new_n4_brain_mask) #MRMask-Brain
    itk.imwrite(im1iso, new_mra_angio) #MRA-ISO
    im1iso.DisconnectPipeline()
    itk.imwrite(im2iso, new_mra_t1) #MRT1-ISO
    itk.imwrite(brain, new_n4_mask) #MRA-Brain
    brain.DisconnectPipeline()
    brainMath.SetInput(im2iso)
    im2iso.DisconnectPipeline()
    brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0 )
    brainMaskD.DisconnectPipeline()
    itk.imwrite(brainMath.GetOutput(), new_im2iso_n4_brain_mask) #MRT1-Brain

    new_file_set = ExtendedFileSet(
        n4_brain_mask=new_n4_brain_mask, #MRMask-Brain
        mra_angio=new_mra_angio, #MRA-ISO
        mra_t1=new_mra_t1, #MRT1-ISO
        n4_mask=new_n4_mask, #MRA-Brain
        im2iso_n4_brain_mask=new_im2iso_n4_brain_mask #MRT1-Brain
    )
    
    return new_file_set

In [10]:
def enhance_and_extract_vessels(file_set: ExtendedFileSet, output_dir: str) -> VessFileSet:
    base_name = file_set.get_base_name()

    # Set output file paths for the new MHA files
    new_im1vess = os.path.join(output_dir, f"{base_name}_MRA-VesselEnhanced_patient.mha")
    new_im1BrainVess = os.path.join(output_dir, f"{base_name}_MRA-Brain-VesselEnhanced_patient.mha")

    ImageType = itk.Image[itk.F, 3]
    ReaderType = itk.ImageFileReader[ImageType]
    ResampleType = ttk.ResampleImage[ImageType]
    
    reader = ReaderType.New(FileName=file_set.mra_angio)
    reader.Update()
    im1iso = reader.GetOutput()
    del reader
    os.remove(file_set.mra_angio)

    reader = ReaderType.New(FileName=file_set.n4_mask)
    reader.Update()
    im1Brain = reader.GetOutput()
    del reader
    os.remove(file_set.n4_mask)

    reader = ReaderType.New(FileName=file_set.mra_t1)
    reader.Update()
    im2iso = reader.GetOutput()
    del reader
    os.remove(file_set.mra_t1)

    reader = ReaderType.New(FileName=file_set.im2iso_n4_brain_mask)
    reader.Update()
    im2Brain = reader.GetOutput()
    del reader
    os.remove(file_set.im2iso_n4_brain_mask)

                 
    imMath = ttk.ImageMath[itk.Image[itk.F,3]].New()
    imMath.SetInput(im1Brain)
    imMath.Blur(1.0)
    imBlur = imMath.GetOutput()
    imBlurArray = itk.GetArrayViewFromImage(imBlur)

    numSeeds = 10
    seedCoverage = 20
    seedCoord = np.zeros([numSeeds,3])
    for i in range(numSeeds):
        seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
        indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
        minX = max(indx[0]-seedCoverage,0)
        maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
        minY = max(indx[1]-seedCoverage,0)
        maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
        minZ = max(indx[2]-seedCoverage,0)
        maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
        imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=0
        indx.reverse()
        seedCoord[:][i] = im1Brain.TransformIndexToPhysicalPoint(indx)
    # Manually extract a few vessels to form an image-specific training set
    vSeg = ttk.SegmentTubes[ImageType].New()
    vSeg.SetInput(im1iso)
    vSeg.SetVerbose(True)
    vSeg.SetMinRoundness(0.1)
    vSeg.SetMinCurvature(0.001)
    vSeg.SetRadiusInObjectSpace( 1 )
    for i in range(numSeeds):
        vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    tubeMaskImage = vSeg.GetTubeMaskImage()
    LabelMapType = itk.Image[itk.UC,3]

    trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
    trMask.SetInput( tubeMaskImage )
    trMask.SetGap( 3 )
    trMask.SetObjectWidth( 1 )
    trMask.SetNotObjectWidth( 1 )
    trMask.Update()
    fgMask = trMask.GetOutput()
    enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
    enhancer.SetInput( im1iso )
    enhancer.AddInput( im2iso )
    enhancer.SetLabelMap( fgMask )
    enhancer.SetRidgeId( 255 )
    enhancer.SetBackgroundId( 128 )
    enhancer.SetUnknownId( 0 )
    enhancer.SetTrainClassifier(True)
    enhancer.SetUseIntensityOnly(True)
    enhancer.SetScales([0.3333,1,2.5])
    enhancer.Update()
    enhancer.ClassifyImages()
    im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))
    
    # Save the new MHA files
    itk.imwrite(im1vess, new_im1vess, compression=True)
    brainMask = itk.imread(file_set.n4_brain_mask, itk.F)
    # Remove the old MHA file from step 2
    os.remove(file_set.n4_brain_mask)
    im1BrainVess = itk.MultiplyImageFilter(Input1 = im1vess, Input2=brainMask)
    itk.imwrite(im1BrainVess, new_im1BrainVess, compression=True) 
    
    new_file_set = VessFileSet(
        im1vess = new_im1vess,
        im1BrainVess = new_im1BrainVess
    )
    
    return new_file_set
    

In [11]:
def convert_mha_to_tre(file_set: VessFileSet, output_dir: str) -> None:
    ImageType = itk.Image[itk.F, 3]
    # read input images from the fileset
    im1iso = itk.imread(file_set.im1vess)
    # Remove the old MHA files from step 3
    os.remove(file_set.im1vess)
    im1BrainVess = itk.imread(file_set.im1BrainVess)
    # Remove the old MHA files from step 3
    os.remove(file_set.im1BrainVess)
    # perform image processing
    imMath = ttk.ImageMath.New(im1BrainVess)
    imMath.MedianFilter(1)
    imMath.Threshold(0.0000000001, 1, 1, 0)
    im1VessMask = imMath.GetOutputShort()
    ccSeg = ttk.SegmentConnectedComponents.New(im1VessMask)
    ccSeg.SetMinimumVolume(4)
    ccSeg.Update()
    im1VessMaskCC = ccSeg.GetOutput()
    imMathSS = ttk.ImageMath.New(im1VessMaskCC)
    imMath.Threshold(1,1,0,1)
    im1VessMaskInv = imMath.GetOutputFloat()
    distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
    distFilter.Update()
    im1SeedRadius = distFilter.GetOutput()
    numSeeds = 20
    vSeg = ttk.SegmentTubes.New(Input=im1iso)
    vSeg.SetMinCurvature(0.0)
    vSeg.SetMinRoundness(0.0)
    vSeg.SetMinRidgeness(0.8)
    vSeg.SetMinLevelness(0.0)
    vSeg.SetRadiusInObjectSpace( 0.75 )
    vSeg.SetBorderInIndexSpace(3)
    vSeg.SetSeedMask( im1SeedRadius )
    vSeg.SetSeedRadiusMask( im1SeedRadius )
    vSeg.SetOptimizeRadius(False)
    vSeg.SetUseSeedMaskAsProbabilities(True)
    vSeg.SetSeedExtractionMinimumProbability(0.4)
    vSeg.ProcessSeeds()
    # write output to the specified directory
    filename = os.path.basename(file_set.im1vess)
    base_name = os.path.splitext(filename)[0]
    output_file = os.path.join(output_dir, f"{base_name}.tre")
    SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
    SOWriter.SetFileName(output_file)
    SOWriter.Update()
    im1VessMask.DisconnectPipeline()
    im1VessMaskCC.DisconnectPipeline()
    im1VessMaskInv.DisconnectPipeline()
    im1SeedRadius.DisconnectPipeline()


In [12]:
def process_file_sets(file_sets, output_dir, max_workers, start_time: float):
    futures = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the executor
        for file_set in file_sets:
            future = executor.submit(process_data_pipeline, file_set, output_dir, start_time)
            futures.append(future)
    return futures

In [13]:
def process_data_pipeline(file_set: FileSet, output_dir: str, start_time: float) -> bool:
    try:
        logging.info("Starting to process data pipeline")
        base_name = file_set.get_base_name()
        logging.info(f"Processing {base_name}")
        
        # Record the start time for the current step
        first_step_start_time = time.monotonic()

        # Convert the files in the FileSet and save them as MHA step 1 files
        logging.info(f"Converting and saving images for {base_name} step 1/4")
        converted_file_set = convert_and_save_images(file_set, output_dir)
        logging.info(f"Elapsed time for {base_name} step 1: {time.monotonic() - first_step_start_time:.2f} seconds")
       
        # Record the start time for the current step
        step_start_time = time.monotonic()

        # Convert the files in the FileSet to ExtendedFileSet and save them as MHA step 2 files, then delete MHA step 1 files
        logging.info(f"Converting files to step 2 for {base_name} step 2/4")
        converted_files_set2 = convert_files_step2(converted_file_set, output_dir)
        logging.info(f"Elapsed time for {base_name} step 2: {time.monotonic() - step_start_time:.2f} seconds")
        
        # Record the start time for the current step
        step_start_time = time.monotonic()

        # Convert the files in the FileSet to VessFileSet and save them as MHA step 3 files, then delete MHA step 2 files
        logging.info(f"Enhancing and extracting vessels for {base_name} step 3/4")
        converted_files_set3 = enhance_and_extract_vessels(converted_files_set2, output_dir)
        logging.info(f"Elapsed time for {base_name} step 3: {time.monotonic() - step_start_time:.2f} seconds")
        
        # Record the start time for the current step
        step_start_time = time.monotonic()

        # Convert the files in the FileSet and save them as TRE files, then delete MHA step 3 files
        logging.info(f"Converting to TRE for {base_name} step 4/4")
        convert_mha_to_tre(converted_files_set3, output_dir)
        logging.info(f"Elapsed time for {base_name} step 4: {time.monotonic() - step_start_time:.2f} seconds")
        
        #Calculate the elapsed time for this specific pipeline
        pipeline_elapsed_time = time.monotonic() - first_step_start_time
        # Calculate the elapsed time for the whole pipeline
        whole_pipeline_elapsed_time = time.monotonic() - start_time

        # Return True if processing is successful
        logging.info(f"Successfully processed {base_name}")
        logging.info(f"Elapsed time for this pipeline ({base_name}): {pipeline_elapsed_time/60:.2f} minutes")
        logging.info(f"Elapsed time for the whole pipeline: {whole_pipeline_elapsed_time/60:.2f} minutes")
        return True

    except Exception as e:
        logging.error(f"Error processing {file_set.get_base_name()}: {e}")
        # Return False if an error occurs during processing
        return False


In [ ]:
# Specify the output directory
output_dir = os.path.join(base_directory, "output")
# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Set file_sets
file_sets = get_filtered_file_sets(base_directory, file_patterns, output_dir)

# Record the start time
start_time = time.monotonic()  

# Process file_sets in batches
futures = process_file_sets(file_sets, output_dir, max_workers, start_time)

# Wait for all futures to complete
results = [future.result() for future in futures]

# Count successful and failed processing
successful_processing_count = results.count(True)
failed_processing_count = results.count(False)
print(f"Successfully processed: {successful_processing_count}")
print(f"Failed to process: {failed_processing_count}")

INFO:root:Skipping file set with base name sub-000_ses-20110101 as the output file already exists
INFO:root:Skipping file set with base name sub-001_ses-20101222 as the output file already exists
INFO:root:Skipping file set with base name sub-002_ses-20110127 as the output file already exists
INFO:root:Skipping file set with base name sub-005_ses-20101215 as the output file already exists
INFO:root:Skipping file set with base name sub-006_ses-20101217 as the output file already exists
INFO:root:Skipping file set with base name sub-007_ses-20101225 as the output file already exists
INFO:root:Skipping file set with base name sub-013_ses-20101220 as the output file already exists
INFO:root:Skipping file set with base name sub-015_ses-20101111 as the output file already exists
INFO:root:Skipping file set with base name sub-021_ses-20101111 as the output file already exists
INFO:root:Skipping file set with base name sub-022_ses-20101011 as the output file already exists
INFO:root:Skipping f

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-078\\ses-20100430\\anat\\sub-078_ses-20100430_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-078\\ses-20100430\\anat\\sub-078_ses-20100430_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-078\\ses-20100430\\anat\\sub-078_ses-20100430_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-078\\ses-20100430\\anat\\sub-078_ses-20100430_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-079\\ses-20100502\\anat\\sub-079_ses-20100502_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-079\\ses-20100502\\anat\\sub-079_ses-20100502_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-079\\ses-20100502\\anat\\sub-0

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-111\\ses-20100208\\anat\\sub-111_ses-20100208_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-111\\ses-20100208\\anat\\sub-111_ses-20100208_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-111\\ses-20100208\\anat\\sub-111_ses-20100208_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-111\\ses-20100208\\anat\\sub-111_ses-20100208_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-112\\ses-20100121\\anat\\sub-112_ses-20100121_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-112\\ses-20100121\\anat\\sub-112_ses-20100121_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-112\\ses-20100121\\anat\\sub-1

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-134\\ses-20111127\\anat\\sub-134_ses-20111127_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-134\\ses-20111127\\anat\\sub-134_ses-20111127_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-134\\ses-20111127\\anat\\sub-134_ses-20111127_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-134\\ses-20111127\\anat\\sub-134_ses-20111127_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-140\\ses-20111208\\anat\\sub-140_ses-20111208_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-140\\ses-20111208\\anat\\sub-140_ses-20111208_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-140\\ses-20111208\\anat\\sub-1

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-164\\ses-20111124\\anat\\sub-164_ses-20111124_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-164\\ses-20111124\\anat\\sub-164_ses-20111124_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-164\\ses-20111124\\anat\\sub-164_ses-20111124_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-164\\ses-20111124\\anat\\sub-164_ses-20111124_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-165\\ses-20111105\\anat\\sub-165_ses-20111105_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-165\\ses-20111105\\anat\\sub-165_ses-20111105_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-165\\ses-20111105\\anat\\sub-1

...

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-412\\ses-20131030\\anat\\sub-412_ses-20131030_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-412\\ses-20131030\\anat\\sub-412_ses-20131030_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-412\\ses-20131030\\anat\\sub-412_ses-20131030_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-412\\ses-20131030\\anat\\sub-412_ses-20131030_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-413\\ses-20131005\\anat\\sub-413_ses-20131005_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-413\\ses-20131005\\anat\\sub-413_ses-20131005_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-413\\ses-20131005\\anat\\sub-4

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-428\\ses-20130629\\anat\\sub-428_ses-20130629_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-428\\ses-20130629\\anat\\sub-428_ses-20130629_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-428\\ses-20130629\\anat\\sub-428_ses-20130629_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-428\\ses-20130629\\anat\\sub-428_ses-20130629_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-429\\ses-20130531\\anat\\sub-429_ses-20130531_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-429\\ses-20130531\\anat\\sub-429_ses-20130531_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-429\\ses-20130531\\anat\\sub-4

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-450\\ses-20141125\\anat\\sub-450_ses-20141125_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-450\\ses-20141125\\anat\\sub-450_ses-20141125_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-450\\ses-20141125\\anat\\sub-450_ses-20141125_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-450\\ses-20141125\\anat\\sub-450_ses-20141125_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-451\\ses-20141129\\anat\\sub-451_ses-20141129_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-451\\ses-20141129\\anat\\sub-451_ses-20141129_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-451\\ses-20141129\\anat\\sub-4

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-466\\ses-20140901\\anat\\sub-466_ses-20140901_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-466\\ses-20140901\\anat\\sub-466_ses-20140901_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-466\\ses-20140901\\anat\\sub-466_ses-20140901_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-466\\ses-20140901\\anat\\sub-466_ses-20140901_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-467\\ses-20140817\\anat\\sub-467_ses-20140817_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-467\\ses-20140817\\anat\\sub-467_ses-20140817_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-467\\ses-20140817\\anat\\sub-4

INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-482\\ses-20140210\\anat\\sub-482_ses-20140210_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-482\\ses-20140210\\anat\\sub-482_ses-20140210_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-482\\ses-20140210\\anat\\sub-482_ses-20140210_desc-angio_N4bfc_brain_mask.nii.gz', n4_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-482\\ses-20140210\\anat\\sub-482_ses-20140210_desc-angio_N4bfc_mask.nii.gz')
INFO:root:FileSet(mra_t1='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-484\\ses-20140116\\anat\\sub-484_ses-20140116_T1w.nii.gz', mra_angio='C:/Users/theod/Downloads/esha_topology_final_data/data/MRA\\sub-484\\ses-20140116\\anat\\sub-484_ses-20140116_angio.nii.gz', n4_brain_mask='C:/Users/theod/Downloads/esha_topology_final_data/data/N4_bfc\\sub-484\\ses-20140116\\anat\\sub-4